In [ ]:
import os
import pandas as pd
from sklearn.datasets import load_files
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

In [ ]:
# read in your file 
comments_path = "CommentsApril2018.csv"
comments_csv = pd.read_csv(comments_path)

In [ ]:
# make a directory of each sectionName 
topic_names = list(set(comments_csv['sectionName']))
for topic in topic_names:
    if not os.path.exists("data/"+topic):
        os.makedirs("data/"+topic)